In [1]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ujan/opt/anaconda3/envs/transformer-test/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/ujan/opt/anaconda3/envs/transformer-test/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/ujan/opt/anaconda3/envs/transformer-test/lib/python3.12/site-packages/ipykernel/ker

In [2]:
#tokenizer = AutoTokenizer.from_pretrained("/home/drdo/Caricatures/models/scan_dummy_tokens_gpt2/checkpoint-40000")
tokenizer = AutoTokenizer.from_pretrained('openai-community/gpt2')

#model = GPT2LMHeadModel.from_pretrained("/home/drdo/Caricatures/models/scan_dummy_tokens_gpt2/checkpoint-40000")
model = GPT2LMHeadModel.from_pretrained('openai-community/gpt2')

model.generation_config.max_length = 256

/Users/ujan/opt/anaconda3/envs/transformer-test/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
max_len = 9
dummy_token = "<empty>"

# command type maps
actions = {
    "walk": "I_WALK",
    "run": "I_RUN",
    "jump": "I_JUMP",
    "look": "I_LOOK",
    "turn": dummy_token,
    dummy_token: dummy_token,
    }

turns = {
    "around": "yyyy",
    "opposite": "yy",
    dummy_token: dummy_token
}

directions = {
    "right": "I_TURN_RIGHT",
    "left": "I_TURN_LEFT",
    dummy_token: dummy_token
}

nums = {
    "twice": "xx",
    "thrice": "xxx",
    dummy_token: dummy_token
}

conjs = ["and", "after", dummy_token]

# command structure
command_structure = {
    0: actions,
    1: turns,
    2: directions,
    3: nums,
    4: conjs,
    5: actions,
    6: turns,
    7: directions,
    8: nums,
}

In [4]:
dataset = load_dataset("scan", "simple", trust_remote_code=True)
column_names = dataset["train"].column_names
input_column = column_names[0]
output_column = column_names[1]

def add_empty_token(x):
    command_str = x[input_column]
    command = command_str.split()
    padded_command = []
    index = 0
    c = 0
    while index < max_len:
        expected_cs = command_structure[index]
        if c < len(command) and command[c] in expected_cs:
            padded_command.append(command[c])
            c += 1
        else:
            padded_command.append(dummy_token)
        index += 1
    
    x[input_column] = ' '.join(padded_command)
    return x

test_dataset = dataset["test"].map(
    add_empty_token,
    batched=False,
    desc="Running tokenizer on dataset",
)


In [5]:
test_dataset[1]

{'commands': 'run <empty> right twice after walk <empty> right twice',
 'actions': 'I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN'}

In [6]:
# generation_mode = GenerationMode.GREEDY_SEARCH

context = 'run <empty> right twice after walk <empty> right twice'
#inputs = tokenizer(context+tokenizer.sep_token, return_tensors="pt")
inputs = tokenizer(context, return_tensors="pt")

output = model.generate(**inputs)[0]
#print(output)
#print(tokenizer.decode(output, skip_special_tokens=False))

#tokenizer.decode(output[0], skip_special_tokens=False).replace(context+tokenizer.sep_token, '')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


12
torch.Size([1, 12, 13, 64])
torch.Size([1, 12, 13, 64])

12
torch.Size([1, 12, 14, 64])
torch.Size([1, 12, 14, 64])

12
torch.Size([1, 12, 15, 64])
torch.Size([1, 12, 15, 64])

12
torch.Size([1, 12, 16, 64])
torch.Size([1, 12, 16, 64])

12
torch.Size([1, 12, 17, 64])
torch.Size([1, 12, 17, 64])

12
torch.Size([1, 12, 18, 64])
torch.Size([1, 12, 18, 64])

12
torch.Size([1, 12, 19, 64])
torch.Size([1, 12, 19, 64])

12
torch.Size([1, 12, 20, 64])
torch.Size([1, 12, 20, 64])

12
torch.Size([1, 12, 21, 64])
torch.Size([1, 12, 21, 64])

12
torch.Size([1, 12, 22, 64])
torch.Size([1, 12, 22, 64])

12
torch.Size([1, 12, 23, 64])
torch.Size([1, 12, 23, 64])

12
torch.Size([1, 12, 24, 64])
torch.Size([1, 12, 24, 64])

12
torch.Size([1, 12, 25, 64])
torch.Size([1, 12, 25, 64])

12
torch.Size([1, 12, 26, 64])
torch.Size([1, 12, 26, 64])

12
torch.Size([1, 12, 27, 64])
torch.Size([1, 12, 27, 64])

12
torch.Size([1, 12, 28, 64])
torch.Size([1, 12, 28, 64])

12
torch.Size([1, 12, 29, 64])
torch.Siz

In [7]:
model.config

GPT2Config {
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.45.0.dev0",
  "use_cache": true,
  "vocab_size": 50257
}

In [8]:
768/12

64.0

In [7]:
context = 'run <empty> right twice after walk <empty> right twice'
inputs = tokenizer(context, return_tensors="pt")
tokenizer.decode(model.generate(**inputs)[0], skip_special_tokens=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'run <empty> right twice after walk <empty> right twice<sep>I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN<|endoftext|>'

In [18]:
testset = test_dataset.select(range(100))

In [19]:
count = 0
model.to("cuda")
bar = tqdm(range(len(testset)))
for example in testset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+tokenizer.sep_token, return_tensors="pt").to("cuda")
    output = model.generate(**inputs)[0].to("cpu")
    output = tokenizer.decode(output, skip_special_tokens=False).replace(command+tokenizer.sep_token, '')
    output = output.replace(tokenizer.eos_token, '')
    if output == label:
        count += 1
    bar.update(1)

print(count/len(testset))

  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

1.0


In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968, 0.88, 